In [1]:
!nvidia-smi

Sun Jan 26 17:33:18 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# ***Fine Tunning IMDB***

### Load and Prepare the Dataset

In [27]:
from datasets import load_dataset
dataset = load_dataset('imdb')

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [29]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [30]:
dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

### Preprocess the Data

In [31]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [32]:
tokenized_datasets["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

### Set Up the Training Arguments

In [33]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    eval_strategy ="epoch",          # Evaluate every epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    num_train_epochs=1,              # Number of training epochs
    weight_decay=0.01,               # Strength of weight decay
)
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_

### Initialize the Model

In [34]:
from transformers import AutoModelForSequenceClassification, Trainer

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Train the Model

In [35]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.195300,0.177772


TrainOutput(global_step=1563, training_loss=0.24168002262225308, metrics={'train_runtime': 3362.3242, 'train_samples_per_second': 7.435, 'train_steps_per_second': 0.465, 'total_flos': 6577776384000000.0, 'train_loss': 0.24168002262225308, 'epoch': 1.0})

### Save the Fine-Tuned Model

In [42]:
# Save the model
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-tokenizer')

('./fine-tuned-tokenizer/tokenizer_config.json',
 './fine-tuned-tokenizer/special_tokens_map.json',
 './fine-tuned-tokenizer/vocab.txt',
 './fine-tuned-tokenizer/added_tokens.json',
 './fine-tuned-tokenizer/tokenizer.json')

# ***ArXiv Project***

In [38]:
import arxiv
import pandas as pd

In [39]:
# Query to fetch AI-related papers
query = 'ai OR artificial intelligence OR machine learning'
search = arxiv.Search(query=query, max_results=10, sort_by=arxiv.SortCriterion.SubmittedDate)

# Fetch papers
papers = []
for result in search.results():
    papers.append({
      'published': result.published,
        'title': result.title,
        'abstract': result.summary,
        'categories': result.categories
    })

# Convert to DataFrame
df = pd.DataFrame(papers)

pd.set_option('display.max_colwidth', None)
df.head(10)

<ipython-input-39-5f6bc63cd8db>:7: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


,published,title,abstract,categories
0,2025-01-23 18:59:55+00:00,Fast3R: Towards 3D Reconstruction of 1000+ Images in One Forward Pass,"Multi-view 3D reconstruction remains a core challenge in computer vision,\nparticularly in applications requiring accurate and scalable representations\nacross diverse perspectives. Current leading methods such as DUSt3R employ a\nfundamentally pairwise approach, processing images in pairs and necessitating\ncostly global alignment procedures to reconstruct from multiple views. In this\nwork, we propose Fast 3D Reconstruction (Fast3R), a novel multi-view\ngeneralization to DUSt3R that achieves efficient and scalable 3D reconstruction\nby processing many views in parallel. Fast3R's Transformer-based architecture\nforwards N images in a single forward pass, bypassing the need for iterative\nalignment. Through extensive experiments on camera pose estimation and 3D\nreconstruction, Fast3R demonstrates state-of-the-art performance, with\nsignificant improvements in inference speed and reduced error accumulation.\nThese results establish Fast3R as a robust alternative for multi-view\napplications, offering enhanced scalability without compromising reconstruction\naccuracy.","[cs.CV, cs.AI, cs.GR, cs.RO]"
1,2025-01-23 18:59:47+00:00,CRPO: Confidence-Reward Driven Preference Optimization for Machine Translation,"Large language models (LLMs) have shown great potential in natural language\nprocessing tasks, but their application to machine translation (MT) remains\nchallenging due to pretraining on English-centric data and the complexity of\nreinforcement learning from human feedback (RLHF). Direct Preference\nOptimization (DPO) has emerged as a simpler and more efficient alternative, but\nits performance depends heavily on the quality of preference data. To address\nthis, we propose Confidence-Reward driven Preference Optimization (CRPO), a\nnovel method that combines reward scores with model confidence to improve data\nselection for fine-tuning. CRPO selects challenging sentence pairs where the\nmodel is uncertain or underperforms, leading to more effective learning. While\nprimarily designed for LLMs, CRPO also generalizes to encoder-decoder models\nlike NLLB, demonstrating its versatility. Empirical results show that CRPO\noutperforms existing methods such as RS-DPO, RSO and MBR score in both\ntranslation accuracy and data efficiency.","[cs.CL, cs.AI, cs.CV]"
2,2025-01-23 18:59:43+00:00,Can We Generate Images with CoT? Let's Verify and Reinforce Image Generation Step by Step,"Chain-of-Thought (CoT) reasoning has been extensively explored in large\nmodels to tackle complex understanding tasks. However, it still remains an open\nquestion whether such strategies can be applied to verifying and reinforcing\nimage generation scenarios. In this paper, we provide the first comprehensive\ninvestigation of the potential of CoT reasoning to enhance autoregressive image\ngeneration. We focus on three techniques: scaling test-time computation for\nverification, aligning model preferences with Direct Preference Optimization\n(DPO), and integrating these techniques for complementary effects. Our results\ndemonstrate that these approaches can be effectively adapted and combined to\nsignificantly improve image generation performance. Furthermore, given the\npivotal role of reward models in our findings, we propose the Potential\nAssessment Reward Model (PARM) and PARM++, specialized for autoregressive image\ngeneration. PARM adaptively assesses each generation step through a potential\nassessment approach, merging the strengths of existing reward models, and\nPARM++ further introduces a reflection mechanism to self-correct the generated\nunsatisfactory image. Using our investigated reasoning strategies, we enhance a\nbaseline model, Show-o, to achieve superior results, with a significant +24%\nimprovement on the GenEval benchmark, surpassing Stable Diffusion 3 by +15%. We\nhope our study provides unique insights an

In [40]:
# Example abstract from API
abstract = df['abstract'][0]

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Summarization
summarization_result = summarizer(abstract)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [41]:
summarization_result[0]['summary_text']

"Multi-view 3D reconstruction remains a core challenge in computer vision. We propose Fast 3D Reconstruction (Fast3R), a novel multi-view generalization to DUSt3R. Fast3R's Transformer-based architecture encompasses N images in a single forward pass, bypassing the need for iterative global alignment."